In [13]:
import pandas as pd
import numpy as np

import acquire
import env

#### <span style="color:red">In writing the function to bring in my data from MySQL, I imported double the number of data points the rest of my team had imported.  Problem solved when dropping the 'USING' command in table joining query.</span>

Some Markdown text with <span style="color:blue">some *blue* text</span>.

### Importing the telco_churn data, 'customers' table, first round.  Just want to get an idea of what columns we're looking at - Round 1

**Different rounds are outlined later, as investigation should lead to the features we'll be needing most for our mvp.**


#### First Hypotheses for customer table:

#### 1.) 

- $H_0$ = contract type has no effect on churn rate
- $H_a$ = contract type DOES have an effect on churn rate

#### 2.) 

- $H_0$ = month-to-month tenure does NOT have an effect on churn rate
- $H_a$ = month-to-month tenure DOES have an effect on churn rate

#### 3.) 

- $H_0$ = contract length of one-year does NOT have an effect on churn rate
- $H_a$ = contract length of one-year DOES have an effect on churn rate

#### 4.)

- $H_0$ = contract length of two years does NOT affect churn rate
- $H_0$ = contract length of two years DOES affect churn rate

In [14]:
telco = acquire.get_telco_data()
telco.head(10)

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.90,542.4,No,Month-to-month,DSL,Mailed check
1,4,1,1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,No,No,No,Yes,Yes,Yes,Yes,69.40,571.45,No,Month-to-month,DSL,Credit card (automatic)
2,1,1,1,0015-UOCOJ,Female,1,No,No,7,Yes,No,Yes,No,No,No,No,No,Yes,48.20,340.35,No,Month-to-month,DSL,Electronic check
3,1,1,1,0023-HGHWL,Male,1,No,No,1,No,No phone service,No,No,No,No,No,No,Yes,25.10,25.1,Yes,Month-to-month,DSL,Electronic check
4,3,1,1,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,Yes,No,No,No,No,No,No,30.50,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic)
5,1,1,1,0067-DKWBL,Male,1,No,No,2,Yes,No,Yes,No,No,No,No,No,Yes,49.25,91.1,Yes,Month-to-month,DSL,Electronic check
6,2,1,1,0076-LVEPS,Male,0,No,Yes,29,No,No phone service,Yes,Yes,Yes,Yes,No,No,Yes,45.00,1242.45,No,Month-to-month,DSL,Mailed check
7,2,1,1,0082-LDZUE,Male,0,No,No,1,Yes,No,No,No,No,No,No,No,Yes,44.30,44.3,No,Month-to-month,DSL,Mailed check
8,1,1,1,0096-BXERS,Female,0,Yes,No,6,Yes,Yes,No,No,No,No,No,No,No,50.35,314.55,No,Month-to-month,DSL,Electronic check
9,2,1,1,0096-FCPUF,Male,0,No,No,30,Yes,Yes,Yes,No,No,No,No,Yes,Yes,64.50,1888.45,No,Month-to-month,DSL,Mailed check


In [15]:
telco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
payment_type_id             7043 non-null int64
internet_service_type_id    7043 non-null int64
contract_type_id            7043 non-null int64
customer_id                 7043 non-null object
gender                      7043 non-null object
senior_citizen              7043 non-null int64
partner                     7043 non-null object
dependents                  7043 non-null object
tenure                      7043 non-null int64
phone_service               7043 non-null object
multiple_lines              7043 non-null object
online_security             7043 non-null object
online_backup               7043 non-null object
device_protection           7043 non-null object
tech_support                7043 non-null object
streaming_tv                7043 non-null object
streaming_movies            7043 non-null object
paperless_billing           7043 non-null object
monthly_charges 

#### No nulls in any of the chosen features.  Sweet.  Lots of encoding, though, because lots of un-integer objects in there.  Namely total_charges is an object when it should probably be a float.  Also, not sure (yet) if all values are unique.

**Also, just from this first pull, the categoricals are:**

- customer_id 
- phone_service ('Yes / No')
- mulitple_lines ('Yes / No')
- internet_service_type_id (1, 2, or 3)
- streaming_tv ('Yes / No') 
- streaming_movies('Yes' / No') 
- paperless_billing ('Yes / No') 
- payment_type_id
- and churn ('Yes / No')

**Leaving the continuous / numerical columns:**

- tenure (months)
- monthly_charges (monetary)
- total_charges (monetary)

In [16]:
telco.shape

(7043, 24)

In [17]:
telco.describe()

,payment_type_id,internet_service_type_id,contract_type_id,senior_citizen,tenure,monthly_charges
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,2.315633,1.872923,1.690473,0.162147,32.371149,64.761692
std,1.148907,0.737796,0.833755,0.368612,24.559481,30.090047
min,1.000000,1.000000,1.000000,0.000000,0.000000,18.250000
25%,1.000000,1.000000,1.000000,0.000000,9.000000,35.500000
50%,2.000000,2.000000,1.000000,0.000000,29.000000,70.350000
75%,3.000000,2.000000,2.000000,0.000000,55.000000,89.850000
max,4.000000,3.000000,3.000000,1.000000,72.000000,118.750000


#### In looking at the original dataset, we discussed how phone (and other) companies cross-sell you on product offerings.  While that does increase the convenience of paying one's bills, it also anchors the customer to the company itself.  For instance, if you have a checking account at a bank, don't be surprised if they start selling you on mortgages and car loans.  

#### With this in mind, we came to the conclusion that if the customers in the dataset had multiple products with the telecomm company, there should be some weight placed on the coefficients of those variables.  Thus, a point system.

In [18]:
# A couple of list comprehensions to put 'Yes' and 'No' values on point system.


phone_service = [0 if i == "No" else 1 for i in telco.phone_service]
mulitple_lines = [1 if i == "Yes" else 0 for i in telco.multiple_lines]

telco["phone_and_multiline"] = [phone_service[i] + mulitple_lines[i] for i in range(len(phone_service))]

#### Below we are implementing a for loop to weigh the family-type of the customers themselves.  The thought behind this being that single parents would be more likely to churn because of financial difficulties - they'd always be on the hunt for better deals.

In [19]:
partner_and_dependents = [] # Married / Cohabitating with kids

for i in range(len(telco.partner)):
    if telco.partner[i] == "No" and telco.dependents[i] == "No":
        partner_and_dependents.append(0)
    elif telco.partner[i] == "Yes" and telco.dependents[i] == "No":
        partner_and_dependents.append(1)
    elif telco.partner[i] == "No" and telco.dependents[i] == "Yes":
        partner_and_dependents.append(2)
    elif telco.partner[i] == "Yes" and telco.dependents[i] == "Yes":
        partner_and_dependents.append(3)

In [20]:
pd.set_option('display.max_columns', None)

In [21]:
telco.shape

(7043, 25)

#### Sunny raised the question: how do we read 'No Internet Service' values.  If they're nulls, can we drop them?  The answer is 'No' because one, they make up too large a percentage of the overall data (nearly 22%) to just drop without any affect on model performance.  Second, upon closer investigation, the phrase 'No internet service' seems to be another way of stating the 'internet_service_type_id' of '3,' which does, in fact, have an influence in who / why people churn.

In [22]:
# Running the cells below for all the columns containing 'No internet service'

telco.online_security.value_counts("No internet service")

No                     0.496663
Yes                    0.286668
No internet service    0.216669
Name: online_security, dtype: float64

In [23]:
pd.crosstab(telco.online_backup, telco.online_security)

online_security,No,No internet service,Yes
online_backup,,,
No,2195,0,893
No internet service,0,1526,0
Yes,1303,0,1126


### Notes in creating the 'prepare.py' file:

<span style="color:red">*Gotta give credit where credit is due: while roles of the functions below were discussed as a team, the actual writing and walkthrough of the functions was done by Chase Thompson.  This has proven a weak area for me in the past, and through Chase's diligence and patience, my learning in how to do this has increased by leaps and bounds.* text</span>

- 'fill_na' function - though the data contained no nulls, there were, indeed, spaces as values.  This function replaces those spaces with np.nan.

- 'dropna' function - after replacing the spaces with 'NaN's (and seeing there were not enough NaN values to have an effect on the performance of our model), this function removes NaNs from the rows where they occur.

- 'phone_lines' function - list comprehension function describing the above section of the notebook outlining why a point system with regards to phone line number was important.

- 'partner_and_dependents' function - for loop function described above assigning weights to family life of individual customers.

- 'drop_columns' function - function to drop the dataframe columns we used to aggregate the values determined in the 'phone_lines' and 'partner_and_dependents' functions.

- 'X_label_encode' function - since ML algorithms can't handle strings or objects as inputs, these columns were label encoded to give them a numeric representation.  Being that LabelEncoder actually assigns a magnitude to the variables, the columns chosen to LabelEncode are considered to be important in determining customer churn rates.

- 'one_hot_encoder' function - Like LableEncoder, OneHot assigns numbers to object variables.  However, the features chosen in this function were deemed 'not as important' to customer churn rates, and OneHot minimizes the impact their magnitude would have over our overall model performance.

- 'drop_service_type' function - another aggregate function to drop the columns used in determining the 'one_hot_encoder' function.

- 'split_telco' function - function that splits telco dataset into train, validate, and test dataframes.
